# TINerator - Complete Workflow

This notebook is intended to demonstrate, more or less comprehensively, the complete workflow of TINerator and advanced functionality.

In [1]:
import tinerator as tin
from tinerator import examples

import numpy as np
from rich import print

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


# 1. Importing GIS Datasets

## 1.1 Loading rasters and shapefiles

In [2]:
dem_filename = examples.new_mexico.dem
watershed_boundary = examples.new_mexico.boundary
watershed_flowlines = examples.new_mexico.flowline

print(f"DEM filepath (GeoTIFF): {dem_filename}\n")
print(f"Watershed DEM boundary filepath (Shapefile): {watershed_boundary}\n")
print(f"Flowline filepath (Shapefile): {watershed_flowlines}\n")

DEM filepath (GeoTIFF): /Users/livingston/dev/lanl/tinerator/tinerator/examples/data/new_mexi
co/rasters/USGS_NED_13_n37w108_Clipped.tif

Watershed DEM boundary filepath (Shapefile): /Users/livingston/dev/lanl/tinerator/tinerator/e
xamples/data/new_mexico/shapefiles/WBDHU12/WBDHU12.shp

Flowline filepath (Shapefile): /Users/livingston/dev/lanl/tinerator/tinerator/examples/data/n
ew_mexico/shapefiles/NHDFlowline/NHDFlowline.shp

In [3]:
pheader = lambda s: print(f"[blue][bold]{s}[/blue][/bold]")
pinfo = lambda s1, s2: print(f"--> {s1}: \n\t{s2}\n")

dem = tin.gis.load_raster(dem_filename)
print(f"Loaded DEM: \n\t{dem}\n")

boundary = tin.gis.load_shapefile(watershed_boundary)
print(f"Loaded boundary: \n\t{boundary}\n")

flowlines = tin.gis.load_shapefile(watershed_flowlines)
print(f"Loaded flowlines: \n\t{flowlines}\n")

Loaded DEM: 
        Raster<data.shape=(2373, 2575), extent=[-107.70482, 36.29657, -107.46639, 36.5163], 
CRS="NAD83">

Loaded boundary: 
        Geometry<"Polygon", shapes=1, crs="NAD83", properties=1>

Loaded flowlines: 
        Geometry<"3D MultiLineString", shapes=3694, crs="NAD83", properties=16>

## 1.2 Plotting GIS objects

In [4]:
tin.plot2d([])#[dem, boundary, flowlines])

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## 1.3 Extracting metadata from GIS objects

### 1.3.1 Raster metadata

In [42]:
pheader("Spatial Information")
pinfo("CRS (Coordinate Reference System)", dem.crs)
pinfo("Geotransform array", dem.geotransform)
pinfo("Raster Extent (min_x, min_y, max_x, max_y)", dem.extent)
pinfo("Raster Centroid", dem.centroid)
pinfo("Cell length/height [bold]value[/bold]", dem.cell_size)
pinfo("Cell length/height [bold]units[/bold]", dem.units)

pheader("Matrix Information")
pinfo("Matrix (np.ndarray) shape", dem.shape)
pinfo("\"No Data\" value", dem.no_data_value)
pinfo("[italic]Maximum[/italic] raster value", np.nanmax(dem.masked_data()))
pinfo("[italic]Minimum[/italic] raster value", np.nanmin(dem.masked_data()))
pinfo("Number of \"No Data\" cells", np.sum(np.isnan(dem.masked_data())))

Spatial Information

--> CRS (Coordinate Reference System): 
        GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.2572
22101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degr
ee",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST
],AUTHORITY["EPSG","4269"]]

--> Geotransform array: 
        (-107.704815288716, 9.259259299999904e-05, 0.0, 36.516296456968, 0.0, 
-9.2592593000001e-05)

--> Raster Extent (min_x, min_y, max_x, max_y): 
        (-107.704815288716, 36.296574233779005, -107.46638936174101, 36.516296456968)

--> Raster Centroid: 
        (-107.5856023252285, 36.4064353453735)

...

--> Matrix (np.ndarray) shape: 
        (2373, 2575)

--> "No Data" value: 
        -3.4028230607370965e+38

--> Maximum raster value: 
        2278.521728515625

--> Minimum raster value: 
        1834.21923828125

--> Number of "No Data" cells: 
        0

### 1.3.2 Shapefile metadata

In [13]:
print(dem.extent)
print(boundary.extent)
print(flowlines.extent)
print(dem.centroid)
print(boundary.centroid)
print(flowlines.centroid)

extents = np.vstack([dem.extent, boundary.extent, flowlines.extent])
print(extents)

(-107.704815288716, 36.296574233779005, -107.46638936174101, 36.516296456968)

(-107.6803268235073, 36.32851344361086, -107.53887725289354, 36.49256139856453)

(-107.74191040986972, 36.27993781035292, -107.43505067701267, 36.54219140994587)

(-107.5856023252285, 36.4064353453735)

[-107.60136606   36.39617767]

[-107.57728379   36.41307757]

[[-107.70481529   36.29657423 -107.46638936   36.51629646]
 [-107.68032682   36.32851344 -107.53887725   36.4925614 ]
 [-107.74191041   36.27993781 -107.43505068   36.54219141]]

[-107.74191040986972,
 36.27993781035292,
 -107.43505067701267,
 36.54219140994587]

In [20]:
import plotly.graph_objects as go
fig = go.Figure()
vars(fig)

{'_validate': True,
 '_grid_str': None,
 '_grid_ref': None,
 '_data_validator': <plotly.validators._data.DataValidator at 0x7fd6efc31400>,
 '_data_objs': [],
 '_data': [],
 '_data_defaults': [],
 '_layout_validator': <plotly.validators._layout.LayoutValidator at 0x7fd6efc31160>,
 '_layout_obj': Layout({
     'template': '...'
 }),
 '_layout': {'template': {'data': {'bar': [{'error_x': {'color': '#2a3f5f'},
      'error_y': {'color': '#2a3f5f'},
      'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
       'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
      'type': 'bar'}],
    'barpolar': [{'marker': {'line': {'color': '#E5ECF6', 'width': 0.5},
       'pattern': {'fillmode': 'overlay', 'size': 10, 'solidity': 0.2}},
      'type': 'barpolar'}],
    'carpet': [{'aaxis': {'endlinecolor': '#2a3f5f',
       'gridcolor': 'white',
       'linecolor': 'white',
       'minorgridcolor': 'white',
       'startlinecolor': '#2a3f5f'},
      'baxis': {'endlinecolor': '#2a3f

## 1.4 GIS pre-processing operations

### 1.4.1 Clipping rasters with shapefiles

### 1.4.2 Reprojecting

### 1.4.3 Changing/adding new No Data values to Rasters

## 1.5 Using Python to create GIS objects

### 1.5.1 Creating Rasters from NumPy matrices

### 1.5.2 Creating Shapefiles from Shapely objects

## 1.6 TINerator GIS toolkit

### 1.6.1 Rasters: pit-filling

### 1.6.2 Rasters: extracting flowlines via watershed delineation

## 1.7 Saving TINerator objects to GIS formats